In [1]:
import gym

#create a single game instance
env = gym.make("Taxi-v2")

#start new game
env.reset();

In [2]:
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [3]:
print("initial observation code:", env.reset())
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)
env.render()

initial observation code: 222
observations: Discrete(500) n= 500
actions: Discrete(6) n= 6
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Random search

In [4]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n

In [5]:
def get_random_policy():
    return np.random.randint(6, size=500)

In [6]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [0.1668264 0.1667818 0.166578  0.166587  0.166508  0.1667188]
Seems fine!


## Let's evaluate!

In [7]:
def sample_reward(env, policy, t_max=100):
    s = env.reset()
    total_reward = 0
    curStep = 0
    is_done = False
    while(not is_done and curStep != t_max): #принудительно заканчиваем игру если за t_max она не закончилась
        s, reward, is_done, _ = env.step(policy[s])
        total_reward += reward
        curStep += 1
    return total_reward

In [8]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env,policy) for _ in range(n_times)]
    return float(np.mean(rewards))        

In [9]:
best_policy = None
best_score = -float('inf')

for i in range(100):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)
        print("Best policy:")

New best score: -663.22
Best policy:
New best score: -583.84
Best policy:
New best score: -546.76
Best policy:
New best score: -511.03
Best policy:
New best score: -494.2
Best policy:
New best score: -466.75
Best policy:
New best score: -420.76
Best policy:


# Part II Genetic algorithm¶

In [10]:
def crossover(policy1, policy2, p=0.5):
    mask = np.random.choice([0, 1], len(policy1), p=[p,1 - p]) == 1
    resPolicy = np.zeros(len(policy1))
    resPolicy[mask] = policy1[mask]
    resPolicy[~mask] = policy2[~mask]
    return resPolicy

In [11]:
def mutation(policy, p=0.5):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(policy, get_random_policy(), p)

In [12]:
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [13]:
n_epochs = 250 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick

In [14]:
np.random.seed(1234)
print("initializing...")
pool = [get_random_policy() for _ in range(pool_size)]
pool_scores = [evaluate(policy) for policy in pool]

initializing...


In [15]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])

In [16]:
for epoch in range(n_epochs):
    print("Epoch %s:"%epoch)
    randNums = np.random.choice(range(len(pool)),  n_crossovers * 2, replace = False)
    crossovered = [crossover(pool[randNums[2*i]], pool[randNums[2*i + 1]]) for  i in range(n_crossovers)]
    
    randNums = np.random.choice(range(len(pool)),  n_mutations, replace = False)
    mutated =  [mutation(pool[randNums[i]]) for  i in range(n_mutations)]
    
    #add new policies to the pool
    pool += crossovered + mutated
    pool_scores = [evaluate(pool[i]) for  i in range(len(pool))]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]
    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])

Epoch 0:
best score: -457.75
Epoch 1:
best score: -439.93
Epoch 2:
best score: -412.84
Epoch 3:
best score: -421.57
Epoch 4:
best score: -377.38
Epoch 5:
best score: -421.93
Epoch 6:
best score: -376.3
Epoch 7:
best score: -395.74
Epoch 8:
best score: -350.92
Epoch 9:
best score: -404.65
Epoch 10:
best score: -341.83
Epoch 11:
best score: -368.2
Epoch 12:
best score: -350.38
Epoch 13:
best score: -358.48
Epoch 14:
best score: -350.92
Epoch 15:
best score: -359.47
Epoch 16:
best score: -287.65
Epoch 17:
best score: -305.29
Epoch 18:
best score: -315.19
Epoch 19:
best score: -306.28
Epoch 20:
best score: -323.38
Epoch 21:
best score: -332.65
Epoch 22:
best score: -314.47
Epoch 23:
best score: -296.38
Epoch 24:
best score: -287.83
Epoch 25:
best score: -270.28
Epoch 26:
best score: -297.19
Epoch 27:
best score: -277.84
Epoch 28:
best score: -252.64
Epoch 29:
best score: -278.02
Epoch 30:
best score: -251.83
Epoch 31:
best score: -260.56
Epoch 32:
best score: -270.19
Epoch 33:
best score: 